## COVID Pipeline Deployment on UCS

### Clone Cisco Kubeflow Starter Pack Repository

In [3]:
! git clone https://github.com/CiscoAI/cisco-kubeflow-starter-pack.git

Cloning into 'cisco-kubeflow-starter-pack'...
remote: Enumerating objects: 1681, done.
remote: Total 1681 (delta 0), reused 0 (delta 0), pack-reused 1681
Receiving objects: 100% (1681/1681), 4.45 MiB | 6.38 MiB/s, done.
Resolving deltas: 100% (661/661), done.


### Import libraries

In [21]:
import kfp
import os
import calendar
import time
from kubernetes import client


### Component files Declarations

In [22]:
path='cisco-kubeflow-starter-pack/apps/healthcare/covid-forecasting/onprem/pipelines/'
component_root_preprocess = path + 'components/covid_preprocess/'
component_root_train = path + 'components/covid_train/'
component_root_serve = path + 'components/covid_serve/'

#### Components Description
covid_datadownload_op &nbsp; - &nbsp; Downloads dataset from GCP bucket and stores in nfs-volume. <br>
covid_preprocess_op &nbsp; - &nbsp; Preprocess the raw dataset to make it suitable for training. <br>
covid_train_op &nbsp; - &nbsp; Load dataset from nfs-volume, train COVID-19 model and saves model in nfs-volume. <br>
covid_serve_op &nbsp; - &nbsp; Load COVID-19 model from nfs-volume and serve the model using tf-serving for later prediction. <br> 
covid_upload_result_op &nbsp; - &nbsp; Uploading the prediction results into GCS bucket for the sake of visualisation. <br> 



### Load components from respective .YAML config files

In [23]:
covid_preprocess_op = kfp.components.load_component_from_file(os.path.join(component_root_preprocess, 'component.yaml'))
covid_train_op = kfp.components.load_component_from_file(os.path.join(component_root_train, 'component.yaml'))
covid_serve_op = kfp.components.load_component_from_file(os.path.join(component_root_serve, 'component.yaml'))

### Define Volume and Volume Mounts

In [24]:
nfs_pvc = client.V1PersistentVolumeClaimVolumeSource(claim_name='nfs')
nfs_volume = client.V1Volume(name='nfs', persistent_volume_claim=nfs_pvc)
nfs_volume_mount = client.V1VolumeMount(mount_path='/mnt/', name='nfs')

### Create Timestamp

In [25]:
timestamp = str(calendar.timegm(time.gmtime()))
print(timestamp)

1588242007



### Define pipeline function

In [26]:
def covid_pipeline():
    
    
    #Define Task for Preprocess covid data
    covid_preprocess_task = covid_preprocess_op()
    covid_preprocess_task.add_volume(nfs_volume)
    covid_preprocess_task.add_volume_mount(nfs_volume_mount)
    
    #Define Task for Training model for COVID-19 prediction
    covid_train_task = covid_train_op(epochs=10)
    covid_train_task.add_volume(nfs_volume)
    covid_train_task.add_volume_mount(nfs_volume_mount)
    covid_train_task.after(covid_preprocess_task)
    
    #Define Task for Serving for COVID-19 prediction
    covid_serve_task = covid_serve_op(timestamp=timestamp)
    covid_serve_task.add_volume(nfs_volume)
    covid_serve_task.add_volume_mount(nfs_volume_mount)
    covid_serve_task.after(covid_train_task)    

### Create & Execute a COVID Forcast Pipeline

In [27]:
#Create a pipeline run
kfp.Client().create_run_from_pipeline_func(covid_pipeline, arguments={})

RunPipelineResult(run_id=361ddb67-faf7-416e-9212-874bc4d87009)